# Assignment 7: Eating a Magic Cake

#### David Ngo
---

#### Note: This assignment is based on Sargent and Stachurski's article on [Optimal Saving.](https://python.quantecon.org/cake_eating_problem.html) 

### Background:

- We have the following utility function: $E\big[\sum_{t=0}^{\infty}\beta^tu(c_t)\big]$. 
- A nice genie has given us a magic cake. 
- The cake's initial size at time 0 is $y_0z_0$, a known number. 
- You can get to eat a piece of any size from the initial cake. 
- The genie waves her magic wand overnight, and she shocks the cake with a multiplicative random variable $z_t$ whose logarithm is independently and identically distributed $N(\mu, \sigma)$. 
- Your goal is to maximize your utility.

---


### Describe this problem as a dynamic program:

What is the state? 
- The state space, denoted $X$ is the size of the cake when we start with $(y_0, z_0)$.
  
  
What is the control? 
- The control, denoted $U$, is the piece of any size from the cake that you choose to eat. So we have $u \in U$ such that $0 < u < 1$.

What is the law of motion? 
- The law of motion represents the transition into the next price.
- This is the the multiplicative random variable or shock $z_t$, whose logarithm is independently and identically distributed $N(\mu, \sigma)$.

What is the reward? 
- The reward $r$ is $r: X \times Y \rightarrow \mathbb{R}$, so we have the reward r(x,y) for $x \in X$ of we choose some fraction $u \in U$.

What is the discount factor? 
- The discount factor is $\beta$, how much we discount to forgo current felicity in favor of future felicity.

---

### Write Bellman's equation for this problem.
- The Bellman equation in this instance is $V(x_t) = max_{0 \leq u \leq 1}\{u(c) + \beta V(x-c)\}$. 
- $V(\cdot)$ represents a value fuction where fore each $c$ given a value $0 \leq u \leq 1$ we maximize the expected value of the optimal plan. 

---

### Problem

- Guess that the optimal policy is to eat a constant fraction of the magic cake in every period. 
- Assume that $u(c) = \frac{c^{1-r}-1}{1-r}$ for $r \geq 0$. 
- Also fix $z_0y_0=10$, $r=2$, $\beta = 0.97$, $\mu=0$, and $\sigma = 0.1$. 
- Using np.random.seed(1234), generate a fixed sequence of 100 log normally distributed random variables. 
- Use Python to search for the fraction that maximizes the expected reward for that fixed sequence. 
- Hint: Refer to Magic Cake from Sargent's Quantecon webpage for aspects of this problem.

---


# Cake eating as a dynamic program
<ol>
    <li> The state space is the random size of the cake</li>  
    <li> The control space is how much one eats in period $t$</li> 
    <li> The law of motion is $x_{t+1} = z_{t+1}(x_t-c_t)$, where $x_t$ is the known size of the cake at time $t$. $z_{t+1}$ is lognormally distributed with parameters $\mu$ and $\sigma$.</li> 
    <li> The reward function is $u(c)$.</li>
    <li> The discount factor is $\beta$. </li>   
</ol>

Bellman's equation is

$$
V(x_t) = max_{c_t \in [0,x_t]}\left\{ u(c_t) + \beta E[V\left(z_{t+1}(x_t-c_t)\right)|z_t] \right\}
$$

I honestly don't know if eating a constant share of the cake makes sense when there are random shocks.  But this porblem set is meant for you to explore these ideas.

In [1]:
import numpy as np

# Finding the fraction that maximizes the expected reward for the fixed sequence

z0y0 = 10
r = 2
beta = 0.97
mu = 0
sigma = .1
T = 100

# Generate a fixed sequence of 100 log normally distributed random variables.
np.random.seed(1234)
sequence = np.random.lognormal(mu, sigma, T)

In [2]:
# Felicity function

def felicity(c,r):
    """""
    
    Computing the felicity given consumption c
    and reward r
    
    """""
    
    # u(c)
    u = (c**(1-r)-1) / (1-r)
    
    return u

In [3]:
# Value function

def bigU(c, beta, r, T):
    """""
    
    Computing the sum of utilities
    C is the sequence of consumption
    beta is the discount factor
    r is the reward curvature
    T is the time period
    
    """""
    
    U = 0
    for t in range(T):
        utility = beta**(t)*felicity(c[t], r)
        U+= utility
    
    return U

In [4]:
# Law of motion

def motion(initial_state, z, share):
    """""
    Computing the sequence of consumption
    z equals a sequence of numbers, representing 
    the random shocks to the size of the cake 
    given the initial state
   
    """""

    c = []
    y_0 = initial_state
    c_0 = share * y_0
    c.append(c_0)
    for t in range(len(z)):
        y_1 = z[t] * (y_0 - c_0)
        c_1 = share*y_1
        c.append(c_1)
        y_0 = y_1
        c_0 = c_1
        
    return c

In [5]:
# Maximize expected reward function

# Initial values
z0y0 = 10
r = 2
beta = 0.97
mu = 0
sigma = .1
T = 100


def max_cake(initial_state, 
             beta, 
             r, 
             mu,
             sigma,
             T, 
             a, 
             b, 
             increment):

    """""
    
    Finding the number in range [a,b] that provides 
    the maximum utility
    initial_state is the size of the cake at state 0
    beta is the discount factor
    r is some given value to compute utility for this case
    T is the period of time
    mu is the given mean
    sigma is the given standard deviation
    initial_state is the size of the cake at state 0
    Given the range a to b, with increments from a to b
    
    """""
    
    # Generate a fixed sequence of T log normally distributed random variables.
    np.random.seed(1234)
    sequence = np.random.lognormal(mu, sigma, T)
    
    # Array of fractions
    control = np.arange(a, b, increment)
    
    # Empty array
    control_utilities = []
    
    # Compute value function for each constant fraction
    for i in range(len(control)):
        c = motion(initial_state, sequence, control[i])
        utility = bigU(c, beta, r, T)
        control_utilities.append(utility)
        
    # Find the fraction that maximizes utility
    print('The optimal plan or best constant fraction is', 
          control[control_utilities.index(max(control_utilities))], 
          'which results in the maximum value', max(control_utilities))

max_cake(z0y0, beta, r, mu, sigma, T, .001, 1, .001)

The optimal plan or best constant fraction is 0.024 which results in the maximum value -227.21792893729372


---

### Conclusion

- Provided that the optimal policy is to eat a constant fraction of the magic cake in every period, we find that 0.024 maximizes the expected reward for the fixed sequence. 
- Since the maximum expected reward is -227.2179, I don't think the genie is giving me a good deal. I imagine that the genie is forcing me to eat $only$ this cake for the rest of my life.

---

### Here's a formula from Sargent for the deterministic case: $1−\beta^{1/r}$

In our case, this number is $0.015$  Sargent's problem uses a slighlty different reward function, and we have a lognormally distributed shock.  The lognormal distribution has mean $exp(\mu + \sigma^2/2)$; in our case, the cake is actually expected to grow at 0.005 per year.

In [9]:
1-.97**(1/2)

0.015114219820389518